<a href="https://colab.research.google.com/github/ferris77/my_notebooks/blob/main/python_alur/05-predict_cars_sold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset to check if cars were sold in a website
The features are: mileage_per_year, model_year and price
The output is: sold (yes or no)
Let's try to predict if a certain car, with given features, will be sold or not accordingly to the price demanded from the owner

In [1]:
import pandas as pd

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"

dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [3]:
mapa = {
    "mileage_per_year" : "milhas_por_ano",
    "model_year" : "ano_do_modelo",
    "price" : "preco",
    "sold" : "vendido"
}

dados = dados.rename(columns = mapa)
dados.head()


,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [4]:
a_trocar = {
    "no" : 0,
    "yes" : 1
}

dados.vendido = dados.vendido.map(a_trocar)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [5]:
from datetime import datetime

ano_atual = datetime.today().year
dados["idade_do_modelo"] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,21
1,1,7843,1998,40557.96,1,23
2,2,7109,2006,89627.50,0,15
3,3,26823,2015,95276.14,0,6
4,4,7935,2014,117384.68,1,7


In [6]:
dados["km_por_ano"] = dados.milhas_por_ano * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,21,35085.22134
1,1,7843,1998,40557.96,1,23,12622.05362
2,2,7109,2006,89627.50,0,15,11440.79806
3,3,26823,2015,95276.14,0,6,43167.32682
4,4,7935,2014,117384.68,1,7,12770.11290


In [7]:
 dados = dados.drop(columns = ["Unnamed: 0", "milhas_por_ano", "ano_do_modelo"], axis = 1)
 dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,21,35085.22134
1,40557.96,1,23,12622.05362
2,89627.50,0,15,11440.79806
3,95276.14,0,6,43167.32682
4,117384.68,1,7,12770.11290


In [8]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados[["vendido"]]

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import numpy as np

SEED = 5
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print(f'Treinaremos com {len(treino_x)} elementos e testaremos com {len(teste_x)} elementos.')

modelo = LinearSVC(max_iter = 1000, dual = True)
modelo.fit(treino_x, treino_y.values.ravel())
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes)*100

print(f'A acurácia foi de {acuracia:.2f}%.')

Treinaremos com 7500 elementos e testaremos com 2500 elementos.
A acurácia foi de 62.80%.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [9]:
#dummy classifier to use as baseline
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y.values.ravel())
dummy_previsoes = dummy_stratified.predict(teste_x)

dummy_stratified_acuracia = accuracy_score(teste_y, dummy_previsoes)*100

print(f'A acurácia dummy_stratified foi de {dummy_stratified_acuracia:.2f}%.')
 

A acurácia dummy_stratified foi de 52.44%.


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [10]:
#dummy classifier to use as baseline
from sklearn.dummy import DummyClassifier

dummy_mostfrequent = DummyClassifier(strategy = 'most_frequent')
dummy_mostfrequent.fit(treino_x, treino_y.values.ravel())
dummy_previsoes = dummy_mostfrequent.predict(teste_x)

dummy_mostfrequent_acuracia = accuracy_score(teste_y, dummy_previsoes)*100

print(f'A acurácia dummy_mostfrequent foi de {dummy_mostfrequent_acuracia:.2f}%.')

A acurácia dummy_mostfrequent foi de 58.00%.


In [11]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados[["vendido"]]

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

SEED = 5
np.random.seed(SEED)

raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print(f'Treinaremos com {len(raw_treino_x)} elementos e testaremos com {len(raw_teste_x)} elementos.')

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x , treino_y.values.ravel())
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes)*100

print(f'A acurácia foi de {acuracia:.2f}%.')

Treinaremos com 7500 elementos e testaremos com 2500 elementos.
A acurácia foi de 77.48%.
